# preprocessing

dev env colab
## Load csv file from Gdrive

In [2025]:
# Mount google drive to read images from colab
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/DevEnv/9321

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/DevEnv/9321


In [0]:
# %cd /content/drive/My Drive/DevEnv/9321
# !wget https://github.com/mysilver/COMP9321-Data-Services/raw/master/20t1/assign3/training.csv
# !wget https://github.com/mysilver/COMP9321-Data-Services/raw/master/20t1/assign3/validation.csv

# Start

In [0]:
import pandas as pd
import numpy as np
import requests
import json

from datetime import datetime

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier



from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from math import sqrt
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

In [0]:
# ref: https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas
pd.options.mode.chained_assignment = None  # default='warn'

In [0]:
train_df=pd.read_csv('./training.csv')
validation_df=pd.read_csv('./validation.csv')
Z_ID = 'z5141401'
DECIMAL_LEN = 2

In [0]:
# train_df

## Processing data


|Column|Value  |
|--|--|
|Budget|keep value, not convert|
|Genres|convert to number of genres|
|Homepage|convert to boolean (has a homepage or not)|
|original_language|keep???|
|release_date|convert to year|


[release_year_rita.png](https://i.loli.net/2020/04/16/Ul1LeRQtr6PDM3d.png)

[release_year_me](https://pic.luoxufeiyan.com/uploads/20200416145052.png)


In [0]:
# Only keep certain columns as training set

# keep_col = ['budget', 'homepage', 'original_language', 'release_date']
# conv_num_list = ['cast', 'crew', 'genres', 'keywords', 'production_companies', 'production_countries', 'spoken_languages']
keep_col = ['budget', 'homepage', 'release_date', 'genres', 'production_companies']
conv_num_list = [ 'crew']
Y_label = 'revenue'


### conv funcs

In [0]:
# convert-to num
def conv_to_num(dataset, modify_col):
    for i in range(len(dataset[modify_col])):
        curr_str = dataset[modify_col][i] 
        curr_len = len(json.loads(curr_str))
        dataset[modify_col][i] = curr_len
    return dataset

In [0]:
# preprocessing-homepage
def proc_homepage(dataset):
    modify_col = 'homepage'
    for i in range(len(dataset[modify_col])):
        curr_str = dataset[modify_col][i] 
        # convert to Ture if movie has a homepage
        curr_len = (type(curr_str) is str)
        dataset[modify_col][i] = curr_len
    return dataset

In [0]:
# preprocessing-release_date
def proc_rls_year(dataset):
    modify_col = 'release_date'
    dataset['release_year'] = dataset[modify_col].copy()
    for i in range(len(dataset[modify_col])):
        curr_str = dataset[modify_col][i] 
        # convert to year only
        curr_len = int(curr_str[:4])
        dataset['release_year'][i] = curr_len
    
    return dataset

In [0]:
# preprocessing-release_month
def proc_rls_mo(dataset):
    modify_col = 'release_date'
    dataset['release_month'] = dataset[modify_col].copy()
    for i in range(len(dataset[modify_col])):
        curr_str = dataset[modify_col][i] 
        # convert to month only
        curr_len = int(curr_str[5:7])
        dataset['release_month'][i] = curr_len
    return dataset

In [0]:
# preprocessing-release_weeks
def proc_rls_weeks(dataset):
    dataset['release_week'] = dataset['release_date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').isocalendar()[1])
    return dataset

In [0]:
# preprocessing-release_days of week
def proc_rls_dow(dataset):
    dataset['release_weekday'] = dataset['release_date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').strftime('%A'))
    return dataset

In [0]:
# get main genre
def proc_main_genre(dataset):
    modify_col = 'genres'
    dataset['main_genre'] = dataset[modify_col].copy()
    for i in range(len(dataset[modify_col])):
        curr_str = dataset[modify_col][i] 
        # get first genre
        try:
            j1 = json.loads(curr_str)
            curr_len = j1[0]['name']
            dataset['main_genre'][i] = curr_len
        except:
            dataset['main_genre'][i] = 'Null'
    return dataset

In [0]:
# get main company id
def proc_main_comp_id(dataset):
    modify_col = 'production_companies'
    dataset['main_comp_id'] = dataset[modify_col].copy()
    for i in range(len(dataset[modify_col])):
        curr_str = dataset[modify_col][i] 
        # get first prod company
        j1 = json.loads(curr_str)
        try:
            curr_len = j1[0]['id']
            dataset['main_comp_id'][i] = curr_len
        except:
            dataset['main_comp_id'][i] = 0
    return dataset

In [0]:
# get main company id
def proc_budget_year_ratio(dataset):
    modify_col = 'budget'
    dataset['budget_year_ratio'] = dataset[modify_col].copy()
    for i in range(len(dataset[modify_col])): 
        # calc ratio
        ratio = round(dataset[modify_col][i] / dataset['release_year'][i], 2)
        dataset['budget_year_ratio'][i] = ratio
    return dataset

In [0]:
# original_language
def proc_orig_lang(dataset):
    all_lang = list(set(dataset['original_language'].values))
    le = LabelEncoder()
    le.fit(all_lang)
    dataset['original_language'] = le.transform(dataset['original_language'])
    return dataset

In [0]:
# budget
def proc_norm(dataset, modify_col):
    
    #norm certain cols

    y_df = dataset[modify_col]
    y_norm = (y_df - y_df.min()) / (y_df.max() - y_df.min())
    dataset[modify_col] = y_norm
    return dataset

In [0]:
# inverse norm
def inverse_norm(orig_col, proc_col):
    # orginal col contains raw data
    inverse_col = proc_col * (orig_col.max() - orig_col.min()) + orig_col.min()
    return inverse_col

In [0]:
def proc_data(dataset):
    # add Y label
    selected_col = keep_col + conv_num_list
    selected_col.append(Y_label)
    proc_dataset = dataset[selected_col]
    # proc_dataset = dataset[keep_col]

    # do preprocessing
    #convert to num
    for i in conv_num_list:
        try:
            proc_dataset = conv_to_num(proc_dataset, i)
        except:
            pass

    # other processing
    proc_dataset = proc_main_genre(proc_dataset)
    proc_dataset = proc_main_comp_id(proc_dataset)
    proc_dataset = proc_rls_year(proc_dataset)
    proc_dataset = proc_rls_weeks(proc_dataset)
    proc_dataset = proc_rls_mo(proc_dataset)
    proc_dataset = proc_rls_dow(proc_dataset)
    proc_dataset = proc_homepage(proc_dataset)
    # calc after release year
    proc_dataset = proc_budget_year_ratio(proc_dataset)
    proc_dataset = proc_norm(proc_dataset, 'budget')

    #norm certain cols
    proc_dataset = proc_norm(proc_dataset, Y_label)    

    # one hog
    # one-hot encode genres
    # one_hot = proc_dataset['main_genre'].str.get_dummies(sep=',')
    # # one-hot encode month variables
    # one_hot_month = pd.get_dummies(proc_dataset['release_month'], prefix='month')
    # # one-hot-encode weekday variable
    one_hot_weekday = pd.get_dummies(proc_dataset['release_weekday'], prefix='weekday')
    # # # one-hot-encode weeks variable
    # one_hot_weeks = pd.get_dummies(proc_dataset['release_week'], prefix='week')

    # proc_dataset = proc_dataset.join(one_hot)
    # proc_dataset = proc_dataset.join(one_hot_month)
    proc_dataset = proc_dataset.join(one_hot_weekday)
    # proc_dataset = proc_dataset.join(one_hot_weeks)

    # remove original rows
    remove_rows = ['main_comp_id', 'release_date', 'genres', 'production_companies', 'main_genre', 'release_month','release_weekday', 'budget']
    proc_dataset = proc_dataset.drop(remove_rows, axis=1)
    return proc_dataset

In [0]:
# def proc_data(dataset):
#     # add Y label
#     selected_col = keep_col + conv_num_list
#     selected_col.append(Y_label)
#     proc_dataset = dataset[selected_col]
#     # proc_dataset = dataset[keep_col]

#     # do preprocessing
#     #convert to num
#     for i in conv_num_list:
#         try:
#             proc_dataset = conv_to_num(proc_dataset, i)
#         except:
#             pass

#     # other processing
#     proc_dataset = proc_norm(proc_dataset, 'budget')
#     proc_dataset = proc_homepage(proc_dataset)
#     proc_dataset = proc_rls_date(proc_dataset)

#     #norm certain cols
#     proc_dataset = proc_norm(proc_dataset, Y_label)    
#     return proc_dataset

#### norm

https://stackoverflow.com/questions/33246316/normalization-in-sci-kit-learn-linear-models

https://stackoverflow.com/questions/44552031/sklearnstandardscaler-can-i-inverse-the-standardscaler-for-the-model-output

### process data

In [2046]:
train_set = proc_data(train_df)
vali_set = proc_data(validation_df)





train_set


,homepage,crew,revenue,release_year,release_week,budget_year_ratio,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday
0,True,153,1.000000,2009,50,117969,0,0,0,0,1,0,0
1,True,32,0.344696,2007,20,149476,0,0,1,0,0,0,0
2,True,155,0.315884,2015,44,121588,0,1,0,0,0,0,0
3,True,217,0.389151,2012,29,124254,0,1,0,0,0,0,0
4,True,132,0.101916,2012,10,129224,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,False,12,0.010797,1991,32,7533,0,0,1,0,0,0,0
2096,True,25,0.028901,2007,7,5979,0,0,0,0,0,0,1
2097,True,1,0.017937,2008,37,7968,1,0,0,0,0,0,0
2098,False,8,0.034580,2008,33,7470,1,0,0,0,0,0,0


In [0]:
# train_set['revenue'][1430:1442]

# Fit models

* https://www.pluralsight.com/guides/non-linear-regression-trees-scikit-learn

* https://github.com/wangye707/PY_DELL/blob/b066ae4bb40273b24d698d66450e4207e3eb6808/git_book-master/chapters/Decision_Tree/decisiontree_regressor.py

In [0]:
keep_col = list(train_set.columns.values)
keep_col.remove(Y_label)

X_train = train_set[keep_col]
Y_train = train_set[Y_label]


X_valid = vali_set[keep_col]
Y_valid = vali_set[Y_label]

### decision tree

In [0]:
def dTree(X_train, Y_train):
    model_dtree = DecisionTreeRegressor()
    # dtree = DecisionTreeRegressor()
    model_dtree.fit(X_train, Y_train)

    return model_dtree

### random forest

In [0]:
def randomForest(X_train, Y_train):
    #RF model
    model_rf = RandomForestRegressor(n_estimators=500, oob_score=True, random_state=100)
    # model_rf = RandomForestRegressor(n_estimators=500)
    model_rf.fit(X_train, Y_train) 

    return model_rf

### Adaboost

In [0]:
def adaBoost(X_train, Y_train):
    model_ab = AdaBoostRegressor(n_estimators=500)
    model_ab.fit(X_train, Y_train)

    return model_ab 

### GBRT

In [0]:
def gbrt(X_train, Y_train):
    model_gbrt = GradientBoostingRegressor(n_estimators=500)
    model_gbrt.fit(X_train, Y_train)

    return model_gbrt

# Evaluate

In [0]:
model_gbrt = gbrt(X_train, Y_train)
model = model_gbrt

In [2054]:
# summary output
lst = [Z_ID]
summary_1_filename = Z_ID + '.PART1.summary.csv'
Y_valid_pred = model.predict(X_valid)
lst.append(round(mean_squared_error(Y_valid,Y_valid_pred), DECIMAL_LEN)) 
lst.append(round(pearsonr(Y_valid, Y_valid_pred)[0], DECIMAL_LEN))
print(lst)
summary_1 = pd.DataFrame([lst], columns =['zid','MSR','correlation']) 
summary_1.to_csv(summary_1_filename, index=False)

['z5141401', 0.01, 0.39]


In [0]:
# detail outputs
output_1_filename = Z_ID + '.PART1.output.csv'

outputs_pred = pd.DataFrame(Y_valid_pred)
outputs_inverse = validation_df.copy()
outputs_inverse['predicted_revenue'] = inverse_norm(validation_df[Y_label], outputs_pred)

outputs_1 = pd.DataFrame(outputs_inverse, columns= ['movie_id', 'predicted_revenue'])
outputs_1.to_csv(output_1_filename, index=False)

In [0]:
# outputs_final_1
# # validation_df['movie_id']
# %pwd

# P2

https://blog.csdn.net/u010900574/article/details/52669072

In [0]:
# Only keep certain columns as training set

# keep_col = ['budget', 'homepage', 'original_language', 'release_date']
# conv_num_list = ['cast', 'crew', 'genres', 'keywords', 'production_companies', 'production_countries', 'spoken_languages']

keep_col = ['budget', 'homepage', 'release_date']
conv_num_list = ['genres',  'keywords']
Y_label = 'rating'


### proc data

Not norm Y-label

In [0]:
def proc_data(dataset):
    # add Y label
    selected_col = keep_col + conv_num_list
    selected_col.append(Y_label)
    proc_dataset = dataset[selected_col]
    # proc_dataset = dataset[keep_col]

    # do preprocessing
    #convert to num
    for i in conv_num_list:
        try:
            proc_dataset = conv_to_num(proc_dataset, i)
        except:
            pass

    # other processing
    proc_dataset = proc_norm(proc_dataset, 'budget')
    proc_dataset = proc_homepage(proc_dataset)
    proc_dataset = proc_rls_date(proc_dataset)

    #Do not norm Y label
    # proc_dataset = proc_norm(proc_dataset, Y_label)    
    return proc_dataset

In [2059]:
train_set = proc_data(train_df)
vali_set = proc_data(validation_df)
train_set

,budget,homepage,release_date,genres,keywords,rating
0,0.623684,True,2009,4,21,3
1,0.789474,True,2007,3,16,3
2,0.644737,True,2015,3,7,3
3,0.657895,True,2012,4,21,3
4,0.684211,True,2012,3,16,3
...,...,...,...,...,...,...
2095,0.039474,False,1991,4,7,2
2096,0.031579,True,2007,3,10,3
2097,0.042105,True,2008,3,3,2
2098,0.039474,False,2008,2,3,3


In [0]:
keep_col = keep_col + conv_num_list

X_train = train_set[keep_col]
Y_train = train_set[Y_label]

X_valid = vali_set[keep_col]
Y_valid = vali_set[Y_label]

### SVM

In [0]:
clf = GradientBoostingClassifier(n_estimators=50, learning_rate=1.0,max_depth=1, random_state=10)
clf.fit(X_train,Y_train)
Y_valid_pred = clf.predict(X_valid)

In [2062]:
# summary output
lst = [Z_ID]
summary_2_filename = Z_ID + '.PART2.summary.csv'
rating_labels = [1,2,3]

lst.append(round(precision_score(Y_valid,Y_valid_pred, average='macro'), DECIMAL_LEN)) 
lst.append(round(recall_score(Y_valid, Y_valid_pred, average='macro'), DECIMAL_LEN))
lst.append(round(accuracy_score(Y_valid, Y_valid_pred), DECIMAL_LEN))
print(lst)
summary_2 = pd.DataFrame([lst], columns =['zid','average_precision','average_recall','accuracy' ]) 
summary_2.to_csv(summary_2_filename, index=False)

['z5141401', 0.6, 0.58, 0.68]


In [0]:
# detail outputs
output_2_filename = Z_ID + '.PART2.output.csv'

outputs_pred = pd.DataFrame(Y_valid_pred)
outputs_inverse = validation_df.copy()
outputs_inverse['predicted_rating'] = inverse_norm(validation_df[Y_label], outputs_pred)

outputs_2 = pd.DataFrame(outputs_inverse, columns= ['movie_id', 'predicted_rating'])
outputs_2.to_csv(output_2_filename, index=False)

In [0]:
# from sklearn import tree, svm, naive_bayes,neighbors
# from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier



# clfs = {'svm': svm.SVC(),\
#         'decision_tree':tree.DecisionTreeClassifier(),
#         'naive_gaussian': naive_bayes.GaussianNB(), \
#         # 'naive_mul':naive_bayes.MultinomialNB(),\
#         'K_neighbor' : neighbors.KNeighborsClassifier(),\
#         'bagging_knn' : BaggingClassifier(neighbors.KNeighborsClassifier(), max_samples=0.5,max_features=0.5), \
#         'bagging_tree': BaggingClassifier(tree.DecisionTreeClassifier(), max_samples=0.5,max_features=0.5),
#         'random_forest' : RandomForestClassifier(n_estimators=50),\
#         'adaboost':AdaBoostClassifier(n_estimators=50),\
#         'gradient_boost' : GradientBoostingClassifier(n_estimators=50, learning_rate=1.0,max_depth=1, random_state=10)
#         }

# def try_different_method(clf):
#     clf.fit(X_train,Y_train)
#     score = clf.score(X_valid,Y_valid)
#     print('the score is :', score)

# for clf_key in clfs.keys():
#     print('the classifier is :',clf_key)
#     clf = clfs[clf_key]
#     try_different_method(clf)